# Extraer variables Tabulares

In [4]:
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import pickle

In [5]:
# Cargar endpoint
infile = open('./data/1_GDB_Layers.pkl','rb')
GDB = pickle.load(infile)
infile.close()

In [9]:
trd = GDB['UNTRD']
trd.shape

(3848, 33)

### Pre Filtro

In [10]:
pre_filter = ['COD_ID', 'FAS_CON_P', 'FAS_CON_S', 'FAS_CON_T', 'TEN_LIN_SE', 'CAP_ELO', 'CAP_CHA', 'CONF',
          'POSTO', 'POT_NOM', 'PER_FER', 'PER_TOT', 'CTMT', 'UNI_TR_S', 'SUB', 'CONJ', 'TIP_TRAFO','DESCR', 'ARE_LOC']

trd = trd[pre_filter]

## Expandir Features a partir de otras layers

### Preparar UCBT

In [12]:
UCBT_ENE = ['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 
            'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']

In [19]:
(GDB['UCBT'])['UCBT_ENE_MED'] = GDB['UCBT'][UCBT_ENE].median(axis=1)
(GDB['UCBT'])['UCBT_ENE_STD'] = GDB['UCBT'][UCBT_ENE].std(axis=1)
(GDB['UCBT'])['UCBT_ENE_MAX'] = GDB['UCBT'][UCBT_ENE].max(axis=1)

In [21]:
## Simplified version of mode to return an integer
def _mode(s):
    return s.mode()[0]

In [22]:
UCBTgTRD = GDB['UCBT'].groupby('UNI_TR_D').agg({
    'CLAS_SUB': _mode,
    'CNAE': _mode, 
    'TIP_CC': _mode, 
    'TEN_FORN': _mode, 
    'GRU_TAR': _mode, 
    'CAR_INST':'sum',
    'ARE_LOC': lambda x: (x == 'NU').sum(),
    'TEN_FORN': _mode,
    'UCBT_ENE_MED':'sum', 
    'UCBT_ENE_STD':'median',
    'UCBT_ENE_MAX': 'max',
    'PAC':'count'
}).reset_index()

In [31]:
# Rename PAC a NCON = Numero de usuarios conectados al transformador
UCBTgTRD.rename(columns= {'PAC':'NCON'}, inplace=True)

# Rename PAC a NCON_URB = Numero de usuarios urbanos conectados al transformador
UCBTgTRD.rename(columns= {'ARE_LOC':'NCON_URB'}, inplace=True)

In [32]:
ucbt_ft = ['UNI_TR_D', 'CLAS_SUB', 'CNAE', 'TIP_CC', 'TEN_FORN', 'GRU_TAR', 'CAR_INST',
           'UCBT_ENE_MED', 'UCBT_ENE_STD', 'UCBT_ENE_MAX', 'NCON', 'NCON_URB']

In [33]:
trd = pd.merge(trd, UCBTgTRD[ucbt_ft], how='left', left_on='COD_ID', right_on='UNI_TR_D')

In [34]:
trd.columns

Index(['COD_ID', 'FAS_CON_P', 'FAS_CON_S', 'FAS_CON_T', 'TEN_LIN_SE',
       'CAP_ELO', 'CAP_CHA', 'CONF', 'POSTO', 'POT_NOM', 'PER_FER', 'PER_TOT',
       'CTMT', 'UNI_TR_S', 'SUB', 'CONJ', 'TIP_TRAFO', 'DESCR', 'ARE_LOC',
       'UNI_TR_D', 'CLAS_SUB', 'CNAE', 'TIP_CC', 'TEN_FORN', 'GRU_TAR',
       'CAR_INST', 'UCBT_ENE_MED', 'UCBT_ENE_STD', 'UCBT_ENE_MAX', 'NCON',
       'NCON_URB'],
      dtype='object')

### Preparar CTMT

In [36]:
CT_ENE = ['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 
          'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']

CT_PNTMT = ['PNTMT_01', 'PNTMT_02', 'PNTMT_03', 'PNTMT_04', 'PNTMT_05', 'PNTMT_06',
            'PNTMT_07','PNTMT_08', 'PNTMT_09', 'PNTMT_10', 'PNTMT_11', 'PNTMT_12']

CT_PNTBT = ['PNTBT_01', 'PNTBT_02', 'PNTBT_03', 'PNTBT_04', 'PNTBT_05', 'PNTBT_06',
            'PNTBT_07','PNTBT_08', 'PNTBT_09', 'PNTBT_10', 'PNTBT_11', 'PNTBT_12']

In [37]:
(GDB['CTMT'])['CTMT_ENE_MED'] = GDB['CTMT'][CT_ENE].median(axis=1)
(GDB['CTMT'])['CTMT_ENE_STD'] = GDB['CTMT'][CT_ENE].std(axis=1)
(GDB['CTMT'])['CTMT_ENE_MAX'] = GDB['CTMT'][CT_ENE].max(axis=1)

(GDB['CTMT'])['CTMT_PNTMT_MED'] = GDB['CTMT'][CT_PNTMT].median(axis=1)
(GDB['CTMT'])['CTMT_PNTMT_STD'] = GDB['CTMT'][CT_PNTMT].std(axis=1)
(GDB['CTMT'])['CTMT_PNTMT_MAX'] = GDB['CTMT'][CT_PNTMT].max(axis=1)

(GDB['CTMT'])['CTMT_PNTBT_MED'] = GDB['CTMT'][CT_PNTBT].median(axis=1)
(GDB['CTMT'])['CTMT_PNTBT_STD'] = GDB['CTMT'][CT_PNTBT].std(axis=1)
(GDB['CTMT'])['CTMT_PNTBT_MAX'] = GDB['CTMT'][CT_PNTBT].max(axis=1)

In [40]:
GDB['CTMT'].rename(columns={'COD_ID':'CTMT'}, inplace=True)

In [41]:
ctmt_ft = ['CTMT','CTMT_ENE_MED','CTMT_ENE_STD','CTMT_ENE_MAX','PERD_A3a', 'PERD_A4', 'PERD_B', 'PERD_MED',
           'PERD_A3a_B', 'PERD_A4_B', 'PERD_B_A3a', 'PERD_B_A4','CTMT_PNTMT_MED','CTMT_PNTMT_STD','CTMT_PNTMT_MAX',
           'CTMT_PNTBT_MED','CTMT_PNTBT_STD','CTMT_PNTBT_MAX']

In [42]:
trd = pd.merge(trd, GDB['CTMT'][ctmt_ft], how='left', on='CTMT')

In [43]:
trd.columns

Index(['COD_ID', 'FAS_CON_P', 'FAS_CON_S', 'FAS_CON_T', 'TEN_LIN_SE',
       'CAP_ELO', 'CAP_CHA', 'CONF', 'POSTO', 'POT_NOM', 'PER_FER', 'PER_TOT',
       'CTMT', 'UNI_TR_S', 'SUB', 'CONJ', 'TIP_TRAFO', 'DESCR', 'ARE_LOC',
       'UNI_TR_D', 'CLAS_SUB', 'CNAE', 'TIP_CC', 'TEN_FORN', 'GRU_TAR',
       'CAR_INST', 'UCBT_ENE_MED', 'UCBT_ENE_STD', 'UCBT_ENE_MAX', 'NCON',
       'NCON_URB', 'CTMT_ENE_MED', 'CTMT_ENE_STD', 'CTMT_ENE_MAX', 'PERD_A3a',
       'PERD_A4', 'PERD_B', 'PERD_MED', 'PERD_A3a_B', 'PERD_A4_B',
       'PERD_B_A3a', 'PERD_B_A4', 'CTMT_PNTMT_MED', 'CTMT_PNTMT_STD',
       'CTMT_PNTMT_MAX', 'CTMT_PNTBT_MED', 'CTMT_PNTBT_STD', 'CTMT_PNTBT_MAX'],
      dtype='object')

### Preparar UNTRS

In [44]:
TRS_ENES = ['ENES_01', 'ENES_02', 'ENES_03', 'ENES_04', 'ENES_05', 'ENES_06',
           'ENES_07', 'ENES_08', 'ENES_09', 'ENES_10', 'ENES_11', 'ENES_12',]

TRS_ENET = ['ENET_01', 'ENET_02', 'ENET_03', 'ENET_04', 'ENET_05', 'ENET_06',
            'ENET_07', 'ENET_08', 'ENET_09', 'ENET_10', 'ENET_11', 'ENET_12',]

In [45]:
GDB['UNTRS']['UNTRS_ENES_MED'] = GDB['UNTRS'][TRS_ENES].median(axis=1)
GDB['UNTRS']['UNTRS_ENES_STD'] = GDB['UNTRS'][TRS_ENES].std(axis=1)
GDB['UNTRS']['UNTRS_ENES_MAX'] = GDB['UNTRS'][TRS_ENES].max(axis=1)

GDB['UNTRS']['UNTRS_ENET_MED'] = GDB['UNTRS'][TRS_ENET].median(axis=1)
GDB['UNTRS']['UNTRS_ENET_STD'] = GDB['UNTRS'][TRS_ENET].std(axis=1)
GDB['UNTRS']['UNTRS_ENET_MAX'] = GDB['UNTRS'][TRS_ENET].max(axis=1)

In [48]:
GDB['UNTRS'].rename(columns={'COD_ID':'UNI_TR_S'}, inplace=True)

In [49]:
untrs_ft = ['UNI_TR_S','POT_NOM','POT_F01','POT_F02','PER_FER','PER_TOT','TIP_TRAFO','UNTRS_ENES_MED','UNTRS_ENES_STD',
           'UNTRS_ENES_MAX','UNTRS_ENET_MED','UNTRS_ENET_STD','UNTRS_ENET_MAX']

In [50]:
trd = pd.merge(trd, GDB['UNTRS'][untrs_ft], how='left', on='UNI_TR_S', suffixes=('','_TRS'))

In [51]:
trd.columns

Index(['COD_ID', 'FAS_CON_P', 'FAS_CON_S', 'FAS_CON_T', 'TEN_LIN_SE',
       'CAP_ELO', 'CAP_CHA', 'CONF', 'POSTO', 'POT_NOM', 'PER_FER', 'PER_TOT',
       'CTMT', 'UNI_TR_S', 'SUB', 'CONJ', 'TIP_TRAFO', 'DESCR', 'ARE_LOC',
       'UNI_TR_D', 'CLAS_SUB', 'CNAE', 'TIP_CC', 'TEN_FORN', 'GRU_TAR',
       'CAR_INST', 'UCBT_ENE_MED', 'UCBT_ENE_STD', 'UCBT_ENE_MAX', 'NCON',
       'NCON_URB', 'CTMT_ENE_MED', 'CTMT_ENE_STD', 'CTMT_ENE_MAX', 'PERD_A3a',
       'PERD_A4', 'PERD_B', 'PERD_MED', 'PERD_A3a_B', 'PERD_A4_B',
       'PERD_B_A3a', 'PERD_B_A4', 'CTMT_PNTMT_MED', 'CTMT_PNTMT_STD',
       'CTMT_PNTMT_MAX', 'CTMT_PNTBT_MED', 'CTMT_PNTBT_STD', 'CTMT_PNTBT_MAX',
       'POT_NOM_TRS', 'POT_F01', 'POT_F02', 'PER_FER_TRS', 'PER_TOT_TRS',
       'TIP_TRAFO_TRS', 'UNTRS_ENES_MED', 'UNTRS_ENES_STD', 'UNTRS_ENES_MAX',
       'UNTRS_ENET_MED', 'UNTRS_ENET_STD', 'UNTRS_ENET_MAX'],
      dtype='object')

In [52]:
trd.shape

(3848, 60)

## Exportar Exit Point

In [54]:
filename = './data/21_ETL_Tabular.pkl'
outfile = open(filename,'wb')
pickle.dump(trd, outfile)
outfile.close()